In [1]:
import os.path as op
import matplotlib.pyplot as plt
from itertools import compress
import time
import logging
import mne
from mne import events_from_annotations
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from itertools import compress
import mne
import mne_nirs
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from mne_nirs.signal_enhancement import enhance_negative_correlation, short_channel_regression

from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair,
                                    scalp_coupling_index)

import ROI
from ROI import different_hb, different_roi
from meta import *
from filter_params import FILTER_DICT
from functions_fnirs import *


fnirs_colors = dict(
  hbo='#C91111',
    hbt= '#5B324B',
  hbr='#004E7C',
  )


C:\Users\Admin\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Тут начинается код

In [130]:
def make_evokeds_roi(smr_epochs, rest_epochs, pick):

    smr_roi_epochs = smr_epochs.copy().pick(pick)
    smr_roi_epochs_info = smr_roi_epochs.info
    rest_roi_epochs = rest_epochs.copy().pick(pick)
    
    evoked_smr = smr_roi_epochs.get_data().mean(axis=0)
    evoked_rest = rest_roi_epochs.get_data().mean(axis=0)
    
    return evoked_smr, evoked_rest, smr_roi_epochs_info

In [170]:
filename = r"C:\Users\Admin\Desktop\IMAGERY-FNIRS\AA\AA_MI"
CONDITION = filename[-2:]
SUBJECT = filename[-5:-3]

DROP_CHANS = [
 'S2_D4 760',
 'S2_D4 850',]

raw_intensity = mne.io.read_raw_nirx(filename, verbose=False)
raw_od = optical_density(raw_intensity) #from row wavelength data
raw_od_shorts = mne_nirs.channels.get_short_channels(raw_od)
raw_od.drop_channels(DROP_CHANS) #we had a non-existent channel

raw_od = mne_nirs.signal_enhancement.short_channel_regression(raw_od)
raw_od = mne_nirs.channels.get_long_channels(raw_od)
raw_od = temporal_derivative_distribution_repair(raw_od) #repairs movement artifacts
raw_haemo = beer_lambert_law(raw_od, ppf=0.1) #from wavelength to HbO\HbR
raw_haemo = raw_haemo.filter(**FILTER_DICT)
raw_haemo = enhance_negative_correlation(raw_haemo)

######################################
ids_target= 'SMR'
ids_rest = 'REST'

events, ids = events_from_annotations(raw_haemo)
ids[ids_target] = 1
ids[ids_rest] = 2

IDS_TO_POP = ["2.0", "33.0", "1.0", "2", "1", "33"]
ids_to_pop = IDS_TO_POP
for i in ids_to_pop:
    popper(ids, i)
    
TMIN  = float(-1.5)
TMAX = float(15.0)
BASELINE = (-1.5, 0.5)
SFREQ = 2

epochs = mne.Epochs(
                    raw=raw_haemo,
                    events=events,
                    event_id=ids,
                    baseline=BASELINE,
                    tmin=TMIN,
                    tmax=TMAX,
                    preload=True,
                    verbose=False,
                    # picks=picks
                )
epochs.resample(SFREQ)

info_left_smz = epochs.copy().pick(different_roi['SMZ'][0])
info_right_smz = epochs.copy().pick(different_roi['SMZ'][2])

rest_epochs = epochs['REST']
smr_epochs = epochs['SMR']

info_hbo_total = rest_epochs.copy().pick_types(fnirs='hbo').info
info_hbr_total = rest_epochs.copy().pick_types(fnirs='hbr').info


Reading 0 ... 7525  =      0.000 ...   739.738 secs...


C:\Users\Admin\AppData\Local\Temp\ipykernel_9608\3563551172.py:18: RuntimeWarning: filter_length (13429) is longer than the signal (7526), distortion is likely. Reduce filter length or filter a longer signal.
  raw_haemo = raw_haemo.filter(**FILTER_DICT)


Used Annotations descriptions: ['1.0', '2.0']
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [200]:
curves_hb = 'hbo'

###EVOKEDS LEFT###
M1_evoked_SMR_left, M1_evoked_REST_left, info_m1_left = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][0])
S1_evoked_SMR_left, S1_evoked_REST_left, info_s1_left = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][1])
SMZ_evoked_SMR_left, SMZ_evoked_REST_left, info_smz_left = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][2]
                                                    )
###EVOKEDS RIGHT###
M1_evoked_SMR_right, M1_evoked_REST_right, info_m1_right = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][3]
                                                    ) 
S1_evoked_SMR_right, S1_evoked_REST_right, info_s1_right = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][4]
                                                    )
SMZ_evoked_SMR_right, SMZ_evoked_REST_right, info_smz_right = make_evokeds_roi(
                                                    smr_epochs=smr_epochs, 
                                                    rest_epochs=rest_epochs,
                                                    pick=different_hb[curves_hb][5]
                                                    )

###EVOKEDS FOR TOPOS###

if curves_hb == 'hbo':
    evoked_SMR, evoked_REST, info_evoked = make_evokeds_roi(
                                                        smr_epochs=smr_epochs, 
                                                        rest_epochs=rest_epochs,
                                                        pick=info_hbo_total['ch_names'])
if curves_hb == 'hbr':
    evoked_SMR, evoked_REST, info_evoked = make_evokeds_roi(
                                                        smr_epochs=smr_epochs, 
                                                        rest_epochs=rest_epochs,
                                                        pick=info_hbr_total['ch_names'])
def relative_measure(arr_target, arr_rest):
    a_rest = np.median(arr_rest[1*SFREQ:4*SFREQ])
    relation = (arr_target - a_rest) / a_rest * 100
    return relation    
    
    
###RELATION LEFT###
relation_M1_left = relative_measure(M1_evoked_SMR_left.mean(axis=0),
                                    M1_evoked_REST_left.mean(axis=0))
relation_S1_left = relative_measure(S1_evoked_SMR_left.mean(axis=0),
                                    S1_evoked_REST_left.mean(axis=0) )
relation_SMZ_left = relative_measure(SMZ_evoked_SMR_left.mean(axis=0),
                                    SMZ_evoked_REST_left.mean(axis=0) )

###RELATION RIGHT###
relation_M1_right = relative_measure(M1_evoked_SMR_right.mean(axis=0),
                                    M1_evoked_REST_right.mean(axis=0))
relation_S1_right = relative_measure(S1_evoked_SMR_right.mean(axis=0),
                                    S1_evoked_REST_right.mean(axis=0) )
relation_SMZ_right = relative_measure(SMZ_evoked_SMR_right.mean(axis=0),
                                    SMZ_evoked_REST_right.mean(axis=0))

C:\Users\Admin\AppData\Local\Temp\ipykernel_9608\2519724848.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  evoked_smr = smr_roi_epochs.get_data().mean(axis=0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9608\2519724848.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  evoked_rest = rest_roi_epochs.get_data().mean(axis=0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9608\2519724848.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  evoked_smr = smr_roi_epochs.get_data().mean(axis=0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9608\2519724848.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  evoked_rest = rest_roi

In [172]:
import matplotlib.ticker as ticker

def set_axis_properties(ax, ylims, tlims, title,
                        xlabel='Time, s', 
                        ylabel='Hb concentration, Δ μM\L',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=True):
    
    ax.set_xlim(tlims[0], tlims[1])
    ax.set_ylim(ylims[0], ylims[1])

    # Set the x and y labels
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)

    # Set the tick parameters
    ax.tick_params(axis='x', labelsize=fontsize)
    ax.tick_params(axis='y', labelsize=fontsize)

    ax.xaxis.set_tick_params(width=linewidth)
    ax.yaxis.set_tick_params(width=linewidth)
    for i in ax.spines.values():
       i.set_linewidth(linewidth)

    ax.set_title(title, fontsize=title_size)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.axvline(x=0, color='black', linestyle='--', linewidth=linewidth)
    ax.axhline(y=0, color='black', linewidth=linewidth)
    
    if legend_flag:
        ax.legend(loc='lower center')

In [173]:
%matplotlib auto

Using matplotlib backend: QtAgg


In [220]:
###PLOT RELATION###


from mpl_toolkits.axes_grid1.inset_locator import inset_axes


fig, axes = plt.subplots(1, 2, figsize=(20, 12))
times = np.arange(TMIN, TMAX, 1/2)
linewidth = 1.5
ylims=(-25000, 50000)
topo_linewidth = 1
pointsize = 20
topo_width = topo_height = '30%'


m1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['M1'][0] ]
s1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['S1'][0] ]
smz_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][0] ]

m1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['M1'][2] ]
s1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['S1'][2] ]
smz_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][2] ]

### RELATION RIGHT ####

rel_line_m1_smr, = axes[0].plot(times, relation_M1_left, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
rel_line_s1_smr, = axes[0].plot(times, relation_S1_left, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
rel_line_smz_smr, = axes[0].plot(times, relation_SMZ_left, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])

fill_1 = axes[0].fill_between([0, 4], ylims[0]/3, ylims[1]/3, 
                         color='#576767', alpha=0.15, 
                         label='Task duration')

set_axis_properties(axes[0],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO relation in LEFT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',
                        xlabel='Time, s', 
                        ylabel='Hb relation SMR/REST, %',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=False
                        )

### TOPO FOR LEFT RELATION PICTURE ###
inset_ax1 = inset_axes(axes[0], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_left_smz.info, 
                     ch_groups=[  m1_group_left, 
                                 smz_group_left,
                                s1_group_left],
                     axes=inset_ax1,
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)

### RELATION RIGHT ###

axes[1].plot(times, relation_M1_right, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
axes[1].plot(times, relation_S1_right, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
axes[1].plot(times, relation_SMZ_right, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])


fill_1 = axes[1].fill_between([0, 4], ylims[0]/3, ylims[1]/3, 
                         color='#576767', alpha=0.15, 
                         label='Task duration')

set_axis_properties(axes[1],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO relation in RIGHT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',

                        xlabel='Time, s', 
                        ylabel='',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=False
                        )

### TOPO FOR RIGHT RELATION PICTURE ###
inset_ax2 = inset_axes(axes[1], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_right_smz.info, 
                     ch_groups=[m1_group_right, 
                                smz_group_right,
                                s1_group_right],
                    axes=inset_ax2, 
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)
             
### LEGEND PARAMS ###
ax_legend = fig.add_axes([0.25, 0.25, 0.5, 0.05])
ax_legend.legend([line_m1_smr, line_s1_smr, line_smz_smr, fill_1], 
                 [f'M1 {curves_hb} relation', 
                  f'S1 {curves_hb} relation', 
                  f'SMZ {curves_hb} relation', 
                 'Task duration'], 
                 loc='center', fontsize=11)
ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    
subplot_params = dict(
 top=0.92,
 bottom=0.11,
 left=0.065,
 right=0.945,
 hspace=0.2,
 wspace=0.35
)

plt.subplots_adjust(**subplot_params)




In [221]:
### PLOT HEMODYNAMICS PER SE ###

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
fig, axes = plt.subplots(1, 2, figsize=(20, 12))
times = np.arange(TMIN, TMAX, 1/2)
linewidth = 1.5
ylims=(-12, 16)
topo_linewidth = 1
pointsize = 20
topo_width = topo_height = '30%'


m1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['M1'][0] ]
s1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['S1'][0] ]
smz_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][0] ]

m1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['M1'][2] ]
s1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['S1'][2] ]
smz_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][2] ]

### LEFT PART ###

line_m1_smr, = axes[0].plot(times, M1_evoked_SMR_left.mean(axis=0)*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
line_s1_smr, = axes[0].plot(times, S1_evoked_SMR_left.mean(axis=0)*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
line_smz_smr, = axes[0].plot(times, SMZ_evoked_SMR_left.mean(axis=0)*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
line_m1_rest, = axes[0].plot(times, M1_evoked_REST_left.mean(axis=0)*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
line_s1_rest, = axes[0].plot(times, S1_evoked_REST_left.mean(axis=0)*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
line_smz_rest, = axes[0].plot(times, SMZ_evoked_REST_left.mean(axis=0)*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')
fill_1 = axes[0].fill_between([0, 4], ylims[0]/3, ylims[1]/3, 
                         color='#576767', alpha=0.15, 
                         label='Task duration')

set_axis_properties(axes[0],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in LEFT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',
                    xlabel='Time, s', 
                    ylabel='Hb concentration, Δ μM\L',  
                    linewidth=1.5, 
                    fontsize=14, 
                    title_size=18,
                    legend_flag=False
                        )
### LEFT HEMO TOPOMAP ###
inset_ax1 = inset_axes(axes[0], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_left_smz.info, 
                     ch_groups=[  m1_group_left, 
                                 smz_group_left,
                                s1_group_left],
                     axes=inset_ax1,
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)

### RIGHT PART ###
axes[1].plot(times, M1_evoked_SMR_right.mean(axis=0)*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
axes[1].plot(times, S1_evoked_SMR_right.mean(axis=0)*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
axes[1].plot(times, SMZ_evoked_SMR_right.mean(axis=0)*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
axes[1].plot(times, M1_evoked_REST_right.mean(axis=0)*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
axes[1].plot(times, S1_evoked_REST_right.mean(axis=0)*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
axes[1].plot(times, SMZ_evoked_REST_right.mean(axis=0)*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')
fill_1 = axes[1].fill_between([0, 4], ylims[0]/3, ylims[1]/3, 
                         color='#576767', alpha=0.15, 
                         label='Task duration')

set_axis_properties(axes[1],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in RIGHT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',

                        xlabel='Time, s', 
                        ylabel='',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=False
                        )

### RIGHT HEMO TOPOMAP ###
inset_ax2 = inset_axes(axes[1], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_right_smz.info, 
                     ch_groups=[m1_group_right, 
                                smz_group_right,
                                s1_group_right],
                    axes=inset_ax2, 
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)
                     
subplot_params = dict(
 top=0.92,
 bottom=0.11,
 left=0.065,
 right=0.945,
 hspace=0.2,
 wspace=0.35
)

### LEGENDS PARAMS ###
ax_legend = fig.add_axes([0.25, 0.25, 0.5, 0.05])
ax_legend.legend([line_m1_smr, line_s1_smr, line_smz_smr,
                 line_m1_rest, line_s1_rest, line_smz_rest, fill_1], 
                 [f'M1/{curves_hb} SMR', f'S1/{curves_hb} SMR', f'SMZ/{curves_hb} SMR', 
                  f'M1/{curves_hb} REST', f'S1/{curves_hb} REST',f'SMZ/{curves_hb} REST',
                 'Task duration'], 
                 loc='center', fontsize=11)
ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.subplots_adjust(**subplot_params)




In [ ]:
conditions_roi_dict = dict(
    ME_LEFT=M1_evoked_SMR_left.mean(axis=0),
    ME_RIGHT=M1_evoked_SMR_right.mean(axis=0),
    MI_LEFT=M1_evoked_SMR_left.mean(axis=0),
    MI_RIGHT=M1_evoked_SMR_right.mean(axis=0),
    TS_LEFT=S1_evoked_SMR_left.mean(axis=0),
    TS_RIGHT=S1_evoked_SMR_right.mean(axis=0),
    TI_LEFT=S1_evoked_SMR_left.mean(axis=0),
    TI_RIGHT=S1_evoked_SMR_right.mean(axis=0),
    SA_LEFT=SMZ_evoked_SMR_left.mean(axis=0),
    SA_RIGHT=SMZ_evoked_SMR_right.mean(axis=0),
    
    rel_ME_LEFT=relation_M1_left,
    rel_ME_RIGHT=relation_M1_right,
    rel_MI_LEFT=relation_M1_left,
    rel_MI_RIGHT=relation_M1_right,
    rel_TS_LEFT=relation_S1_left,
    rel_TS_RIGHT=relation_S1_right,
    rel_TI_LEFT=relation_S1_left,
    rel_TI_RIGHT=relation_S1_right,
    rel_SA_LEFT=relation_SMZ_left,
    rel_SA_RIGHT=relation_SMZ_right,
    )

def get_max_time(cond_left, cond_right):
    mean_left_cond = conditions_roi_dict[cond_left]
    mean_right_cond = conditions_roi_dict[cond_right]

    peak_timestamp_left = np.argmax(mean_left_cond)
    peak_timestamp_right = np.argmax(mean_right_cond)
    
    max_value_left = mean_left_cond[peak_timestamp_left]
    max_value_right = mean_right_cond[peak_timestamp_right]

    prevalent_side = 'left' if max_value_left > max_value_right else 'right'
    final_timestamp = peak_timestamp_left if max_value_left > max_value_right else peak_timestamp_right
    
    return final_timestamp, prevalent_side

In [254]:
final_timestamp, prevalent_side = get_max_time(f'{CONDITION}_LEFT', f'{CONDITION}_RIGHT')
rel_final_timestamp, rel_prevalent_side = get_max_time(f'rel_{CONDITION}_LEFT', 
                                                       f'rel_{CONDITION}_RIGHT')

In [255]:
smr_in_peak_timestamp = evoked_SMR[:, final_timestamp]
rest_in_peak_timestamp = evoked_REST[:, final_timestamp]
top_chans_SMR = np.argpartition(smr_in_peak_timestamp, -3)[-3:]
top_chans_REST = np.argpartition(rest_in_peak_timestamp, -3)[-3:]
mask_SMR = np.zeros(smr_in_peak_timestamp.shape[0], dtype=bool)
mask_SMR[top_chans_SMR] = True
mask_REST = np.zeros(rest_in_peak_timestamp.shape[0], dtype=bool)
mask_REST[top_chans_REST] = True




In [256]:
from matplotlib.colors import LinearSegmentedColormap
color_list = [(0, fnirs_colors['hbr']), (0.5, '#FFFFE4'), (1, fnirs_colors['hbo'])]
custom_cmap = LinearSegmentedColormap.from_list('Custom_cmap', color_list, )

mask_params = dict(marker='o', markerfacecolor='black', markeredgecolor='white',
                linewidth=0, markersize=6)

ylims=(-8, 8)
fig, axes = plt.subplots(1, 2, figsize=(20, 12))

### LEFT SMR TOPO ###
mne.viz.plot_topomap(data=smr_in_peak_timestamp*1e6,
                        pos=info_hbo_total,
                        axes=axes[0],
                        vlim=ylims,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap, 
                         mask=mask_SMR,
                         mask_params=mask_params)
### RIGHT REST TOPO ###
mne.viz.plot_topomap(data=rest_in_peak_timestamp*1e6,
                        pos=info_hbo_total,
                        axes=axes[1],
                        vlim=ylims,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap,
                        mask=mask_REST,
                        mask_params=mask_params, )

### TITLES ###
axes[0].set_title(
    f'Topography of {curves_hb} in subject {SUBJECT}\n in {CONDITION} condititon, t={int(peak_timestamp/2)}s',
fontsize=18)

axes[1].set_title(
    f'Topography of {curves_hb} in subject {SUBJECT}\n in REST condititon, t={int(peak_timestamp/2)}s',
fontsize=18)

### COLORBAR SETTINGS ###
sm = plt.cm.ScalarMappable(cmap=custom_cmap, 
                           norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))

cbaxes = fig.add_axes([0.075, 0.25, 0.02, 0.5]) # setup colorbar axes. 
cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')

cbar.set_label(' Hb concentration, Δ μM\L', loc='center', size=12)
cbar.ax.yaxis.set_label_coords(-0.5, 0.5)

subplot_params=dict(top=0.89,
bottom=0.21,
left=0.115,
right=0.98,
hspace=0.175,
wspace=0.0)
plt.subplots_adjust(**subplot_params)

### TOP CHANNELS ###

chnames_top_SMR = [i for (i, v) in zip(info_hbo_total['ch_names'], mask_SMR) if v]
chnames_top_REST = [i for (i, v) in zip(info_hbo_total['ch_names'], mask_REST) if v]


In [265]:
from matplotlib.colors import LinearSegmentedColormap
color_list = [(0, fnirs_colors['hbr']), (0.5, '#FFFFE4'), (1, fnirs_colors['hbo'])]
custom_cmap = LinearSegmentedColormap.from_list('Custom_cmap', color_list, )

mask_params = dict(marker='o', markerfacecolor='black', markeredgecolor='white',
                linewidth=0, markersize=6)

ylims=(-8000, 8000)
fig, axes = plt.subplots(1, 2, figsize=(20, 12))


def relative_measure_topo(arr_target, arr_rest):
    a_rest = np.median(arr_rest[:, 1*SFREQ:4*SFREQ], axis=1)
    relation = (arr_target - a_rest[:, None]) / a_rest[:, None] * 100
    return relation

topo_rel = relative_measure_topo(evoked_SMR, evoked_REST)
smr_in_peak_timestamp = topo_rel[:, rel_final_timestamp]

rel_top_chans_SMR = np.argpartition(smr_in_peak_timestamp, -3)[-3:]
rel_mask_SMR = np.zeros(smr_in_peak_timestamp.shape[0], dtype=bool)
rel_mask_SMR[top_chans_SMR] = True

### LEFT SMR TOPO ###
mne.viz.plot_topomap(
                        data=smr_in_peak_timestamp,
                        pos=info_hbo_total,
                        axes=axes[0],
                        vlim=ylims,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap,
                        mask=rel_mask_SMR,
                        mask_params=mask_params
)


### TITLES ###
axes[0].set_title(
    f'Topography of {curves_hb} in subject {SUBJECT}\n in {CONDITION} condititon, t={int(peak_timestamp/2)}s',
fontsize=18)


### COLORBAR SETTINGS ###
sm = plt.cm.ScalarMappable(cmap=custom_cmap, 
                           norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))

cbaxes = fig.add_axes([0.075, 0.25, 0.02, 0.5]) # setup colorbar axes. 
cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')

cbar.set_label(' Hb concentration, Δ μM\L', loc='center', size=12)
cbar.ax.yaxis.set_label_coords(-0.5, 0.5)

# subplot_params=dict(top=0.89,
# bottom=0.21,
# left=0.115,
# right=0.98,
# hspace=0.175,
# wspace=0.0)
# plt.subplots_adjust(**subplot_params)

### TOP CHANNELS ###

# chnames_top_SMR = [i for (i, v) in zip(info_hbo_total['ch_names'], mask_SMR) if v]
# chnames_top_REST = [i for (i, v) in zip(info_hbo_total['ch_names'], mask_REST) if v]


In [266]:
smr_in_peak_timestamp

array([-2.86874488e+02, -6.82978200e+02, -5.59217746e+03,  3.38964543e+03,
       -5.07968292e+03, -3.87706426e+03, -2.51443769e+03, -1.21496536e+04,
       -2.82491131e+02, -7.60115079e+01, -2.87158773e+03, -2.01157890e+03,
        3.08922037e+03, -1.18606120e+03,  8.29785175e+02, -5.83765829e+02,
       -1.53685305e+03, -9.53425986e+02, -3.80280424e+01, -3.52123189e+00,
       -3.87863711e+03, -3.56807765e+02, -2.16474737e+02, -2.64831205e+02,
       -3.84871678e+02, -1.20559623e+03, -2.36588117e+03, -2.82881996e+02,
       -5.53482604e+04, -4.52723056e+02, -7.91907866e+02, -1.01105186e+03,
       -5.14031019e+02, -5.73181292e+02, -9.79613087e+02, -7.37552793e+02,
       -9.95998524e+02, -1.24802915e+03, -1.47853793e+03, -2.22512982e+03,
       -8.22317318e+02, -1.02795470e+03, -1.32526922e+03, -8.92422543e+02,
       -1.88911261e+03, -2.05514569e+04, -9.75405578e+02, -9.10882110e+02,
       -1.37301230e+04, -1.17718545e+03, -1.24307504e+03, -4.17583911e+02,
       -9.64506234e+02, -

In [268]:
topo_rel.shape

(102, 33)

In [ ]:
from matplotlib.colors import ListedColormap
color_list_mine = list(fnirs_colors.values())
cmap_cool = ListedColormap(color_list_mine)

@verbose
def plot_sensors(
    info,
    kind="topomap",
    ch_type=None,
    title=None,
    show_names=False,
    ch_groups=None,
    to_sphere=True,
    axes=None,
    block=False,
    show=True,
    sphere=None,
    pointsize=None,
    linewidth=2,
    *,
    cmap=cmap_cool,
    verbose=None,
):